In [61]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os

acropolis_directory = "/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS/Software/quickflow/acropolis/"

In [62]:
df = pl.scan_parquet(os.path.join(acropolis_directory, "pivot_measurements.parquet"))

df = df.sort("creation_timestamp")
df.tail(3).collect()

system_name,revision,creation_timestamp,receipt_timestamp,wxt532_direction_min,wxt532_speed_min,wxt532_last_update_time,wxt532_speed_avg,wxt532_direction_avg,wxt532_direction_max,wxt532_speed_max,gmp343_compensated,bme280_humidity,sht45_temperature,bme280_pressure,sht45_humidity,gmp343_raw,bme280_temperature,gmp343_filtered,gmp343_temperature,enclosure_bme280_humidity,raspi_cpu_usage,enclosure_bme280_pressure,raspi_cpu_temperature,raspi_disk_usage,raspi_memory_usage,enclosure_bme280_temperature,cal_gmp343_compensated,cal_gmp343_raw,cal_bme280_temperature,cal_gmp343_filtered,cal_sht45_temperature,cal_bme280_pressure,cal_sht45_humidity,cal_gmp343_temperature,cal_bme280_humidity,cal_bottle_id
str,i32,"datetime[ns, UTC]","datetime[ns, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""tum-esm-midcos…",14,2023-10-13 14:23:12.940 UTC,2023-10-13 14:23:13.416742 UTC,null,null,null,null,null,null,null,407.0,15.15,32.77,941.18,17.95,352.0,32.7,407.6,34.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""tum-esm-midcos…",18,2023-10-13 14:23:13.010 UTC,2023-10-13 14:23:13.706551 UTC,null,null,null,null,null,null,null,409.7,15.33,31.12,935.63,19.68,360.8,31.04,405.0,32.6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""tum-esm-midcos…",26,2023-10-13 14:23:15.160 UTC,2023-10-13 14:23:15.761647 UTC,null,null,null,null,null,null,null,423.5,16.02,30.18,931.24,20.14,372.2,29.79,426.0,31.1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [63]:
df = df.with_columns([pl.when(pl.col("cal_gmp343_filtered") < 600).then(pl.col("cal_gmp343_filtered")).otherwise(None).alias("cal_400"),
                      pl.when(pl.col("cal_gmp343_filtered") > 600).then(pl.col("cal_gmp343_filtered")).otherwise(None).alias("cal_800"),
                      pl.when(pl.col("cal_gmp343_filtered") < 600).then(pl.col("cal_bottle_id")).otherwise(None).alias("cal_bottle_id_400"),
                      pl.when(pl.col("cal_gmp343_filtered") > 600).then(pl.col("cal_bottle_id")).otherwise(None).alias("cal_bottle_id_800")])

In [64]:
dfg = df.groupby([pl.col("creation_timestamp").dt.date(), pl.col("system_name")]).agg([pl.col("cal_400").drop_nulls(),
                                                                                    pl.col("cal_800").drop_nulls(),
                                                                                    pl.col("cal_bottle_id_400").drop_nulls().median(),
                                                                                    pl.col("cal_bottle_id_800").drop_nulls().median()])

In [66]:
def average_bottle(data):
    data = data.to_list()
    #2nd bottle
    if 50 < len(data) < 70:
        x = data[int(len(data)*0.3):int(len(data)*0.95)]
        return sum(x) / len(x)
    #1st bottle
    elif 70 < len(data) > 130:
        x = data[int(len(data)*0.5):int(len(data)*0.95)]
        return sum(x) / len(x)
    else:
        return 0.0

dfg.select([pl.col("creation_timestamp"),
            pl.col("cal_400").apply(lambda x: average_bottle(x)).alias("mean_cal_400"),
            pl.col("cal_800").apply(lambda x: average_bottle(x)).alias("mean_cal_800"),
            pl.col("cal_bottle_id_400"),
            pl.col("cal_bottle_id_800")
            ]).collect()

creation_timestamp,mean_cal_400,mean_cal_800,cal_bottle_id_400,cal_bottle_id_800
date,f64,f64,f64,f64
2023-09-21,0.0,0.0,null,null
2023-09-21,0.0,0.0,null,null
2023-09-21,0.0,0.0,null,null
2023-09-21,0.0,0.0,null,null
2023-09-22,0.0,0.0,null,null
2023-09-22,0.0,0.0,null,null
2023-09-23,0.0,0.0,null,null
2023-09-23,0.0,0.0,null,null
2023-09-23,0.0,0.0,null,null
